In [49]:
# In this notebook, we will evaluate the responses generated by the LLMs.

import pandas as pd
import numpy as np
import os
import json
from tqdm.auto import tqdm
import ast


In [50]:
df_questions = pd.read_csv('data/habermas_machine_questions_with_responses.csv')
df_questions['own_opinion.text'] = df_questions['own_opinion.text'].apply(ast.literal_eval)
df_questions.head()

,question.text,own_opinion.text,question_topic,question_id,gpt-4o-mini,gpt-3.5-turbo,gpt-4o
0,Are laws banning private education a good idea?,[I am indifferent regarding this statement for...,70,7,The debate over whether laws banning private e...,Banning private education completely may not b...,The question of whether laws banning private e...
1,"If we could extend human life by 50 years, sho...",[I think we should always advance science and ...,90,128,The question of whether we should extend human...,This question is highly subjective and there a...,Extending human life by 50 years is a complex ...
2,Is it OK to use animal products at all for bea...,[I do not think it is ok to use animal product...,5,157,The use of animal products in beauty care is a...,This is a personal choice and ultimately depen...,The use of animal products in beauty care is a...
3,Is the use of fossil fuels doing more harm tha...,[This question is if you believe in man- made ...,39,343,The impact of fossil fuels on the economy is a...,The use of fossil fuels has both positive and ...,The use of fossil fuels is a complex issue wit...
4,Should parents have a legal right to override ...,[Parents should have a legal right to some deg...,61,511,The question of whether parents should have a ...,There is no simple answer to this question as ...,The question of whether parents should have a ...


In [40]:
# Now we want to see if the opinions are all represented in the generated responses.
from openai import OpenAI
client = OpenAI()

def is_my_opinion_represented(question, response, opinion):
    """
    Determine if the following opinion is represented in the response to a question. This is the simplest approach I can think of.
    """
    prompt = f"""Task: Determine if the following opinion is represented in the response to a question.

Question: {question}
Response: {response}
Opinion to check for: {opinion}

Instructions:
- Answer ONLY with 'yes' or 'no'
- Answer 'yes' if the opinion is clearly represented
- Answer 'no' if the opinion is absent
- Do not explain your reasoning
- Do not add any other text

Answer:"""

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a precise evaluator that only responds with 'yes' or 'no'."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,  # Use 0 for maximum consistency
        max_tokens=1, # We only need one token for 'yes' or 'no'
    )
    
    return completion.choices[0].message.content.strip().lower()

In [ ]:
def is_my_opinion_represented_with_cot_michiel(question, response, opinion):
    """
    Determine if the following opinion is represented in the response to a question. Using a CoT with XML formatting (task for Michiel Bakker).
    """
    return False

In [123]:
from pydantic import BaseModel

class ThinkingStep(BaseModel):
    step_number: int
    thought: str

class RepresentationChain(BaseModel):
    steps: list[ThinkingStep]
    relevance_score: int

def is_my_opinion_represented_structured_cot(question, response, opinion):
    """
    Determine if the opinion is represented in the response to a question, using structured CoT generation.
    """
    system_prompt = f"""Task: Determine if the opinion is represented in the response to the question. 
You should think step-by-step about the question and response to determine if the opinion is represented and then give a score between 0 and 10 of how well it is represented.
"""
    
    prompt ="""Question: {question}
Response: {response}
Opinion to check for: {opinion}
"""

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a precise evaluator that only responds with 'yes' or 'no'."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,  # Use 0 for maximum consistency
        response_format={
            'type': 'json_schema',
            'json_schema': 
                {
                "name": "RepresentationChain", 
                "schema": RepresentationChain.model_json_schema()
                }
            } 
    )
    
    result_object = json.loads(completion.choices[0].message.content)
    return result_object

def process_representation_result(result_object):
    return result_object['relevance_score']


In [124]:
from pydantic import BaseModel

class ThinkingStep(BaseModel):
    step_number: int
    thought: str

class EntailmentJSON(BaseModel):
    matches: list[str]

class EntailmentChain(BaseModel):
    steps: list[ThinkingStep]
    final_answer: EntailmentJSON

def entailment_from_gpt_json(question: str, response: str, opinion: str):
    """
    Find exact text matches between rich text and opinion using GPT-4.

    Args:
        question: Context question or query
        response: Source text to find matches in
        opinion: Opinion text to match against

    Returns:
        Dictionary containing matches as an array of strings
    """
    system_prompt = f"""Task: Text Entailment. Find the exact words in the Rich Text question response that specifically represent the content expressed in the Opinion.

Instructions:
1. For each concept in the Opinion, find the sentences in the Rich Text that specifically represent the content expressed in the Opinion. Think through this step-by-step.
2. Return the EXACT text from the Rich Text that represents the content expressed in the Opinion.

Important:
- Do not paraphrase or interpret - use the exact text.
- Each match must be a continuous span of text.
"""

    prompt = f"""Context question: {question}
Rich Text: {response}
Opinion: {opinion}

Find the text that is entailed by the opinion."""

    chat_response = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0,  # Use 0 for maximum consistency
        response_format={
            'type': 'json_schema',
            'json_schema': 
                {
                "name": "EntailmentChain", 
                "schema": EntailmentChain.model_json_schema()
                }
            } 
    )

    result_object = json.loads(chat_response.choices[0].message.content)
    return result_object

def process_entailment_result(result_object, response):
    matches = []
    for match in result_object['final_answer']['matches']:
        start_index = response.lower().find(match.lower())
        end_index = start_index + len(match)
        matches.append((start_index, end_index))
    return matches


In [128]:
models = ['gpt-4o-mini', 'gpt-3.5-turbo','gpt-4o']

overton_results = []
for _, row in tqdm(df_questions.iterrows(), total=df_questions.shape[0]):
    question = row['question.text']
    opinions = row['own_opinion.text']
    question_id = row['question_id']
    for opinion_idx, opinion in enumerate(opinions):
        for model in models:
            response = row[model]
            result_opinion_represented = is_my_opinion_represented(question, response, opinion)
            result_structured_cot = is_my_opinion_represented_structured_cot(question, response, opinion)
            entailment_result = entailment_from_gpt_json(question, response, opinion)
            entailment_matches = process_entailment_result(entailment_result, response)

            overton_results.append({
                'question_id': question_id,
                'opinion_idx': opinion_idx,
                'model': model,
                'is_represented_simple_prompt': result_opinion_represented == 'yes',
                'is_represented_structured_cot': result_structured_cot,
                'is_represented_structured_cot_score': process_representation_result(result_structured_cot),
                'entailment_result': entailment_result,
                'entailment_matches': entailment_matches
            })

df_overton_results = pd.DataFrame(overton_results)

In [127]:
df_overton_results

,question_id,opinion_idx,model,is_represented_simple_prompt,is_represented_structured_cot,is_represented_structured_cot_score,entailment_result,entailment_matches
0,7,0,gpt-4o-mini,False,"{'steps': [{'step_number': 1, 'thought': 'Iden...",7,"{'steps': [{'step_number': 1, 'thought': 'The ...",[]
1,7,0,gpt-3.5-turbo,False,"{'steps': [{'step_number': 1, 'thought': 'Iden...",0,"{'steps': [{'step_number': 1, 'thought': 'The ...","[(0, 75)]"
2,7,0,gpt-4o,False,"{'steps': [{'step_number': 1, 'thought': 'Iden...",0,"{'steps': [{'step_number': 1, 'thought': 'The ...",[]
3,7,1,gpt-4o-mini,True,"{'steps': [{'step_number': 1, 'thought': 'Iden...",7,"{'steps': [{'step_number': 1, 'thought': 'The ...","[(1424, 1545), (1926, 2071), (1676, 1819)]"
4,7,1,gpt-3.5-turbo,False,"{'steps': [{'step_number': 1, 'thought': 'Iden...",8,"{'steps': [{'step_number': 1, 'thought': 'The ...","[(0, 99), (104, 186)]"
5,7,1,gpt-4o,True,"{'steps': [{'step_number': 1, 'thought': 'Iden...",7,"{'steps': [{'step_number': 1, 'thought': 'The ...","[(1155, 1301), (253, 390), (1343, 1469)]"
6,7,2,gpt-4o-mini,True,"{'steps': [{'step_number': 1, 'thought': 'Iden...",8,"{'steps': [{'step_number': 1, 'thought': 'The ...","[(-1, 225), (-1, 255)]"
7,7,2,gpt-3.5-turbo,False,"{'steps': [{'step_number': 1, 'thought': 'Iden...",7,"{'steps': [{'step_number': 1, 'thought': 'The ...","[(0, 187)]"
8,7,2,gpt-4o,False,"{'steps': [{'step_number': 1, 'thought': 'Iden...",0,"{'steps': [{'step_number': 1, 'thought': 'The ...","[(-1, 162), (1343, 1469), (1581, 1660)]"
9,7,3,gpt-4o-mini,False,"{'steps': [{'step_number': 1, 'thought': 'Iden...",0,"{'steps': [{'step_number': 1, 'thought': 'The ...","[(-1, 225), (-1, 255)]"


In [126]:
df_overton_results = pd.DataFrame(overton_results)

In [68]:
df_overton_results['represented_bool'] = df_overton_results['represented'] == 'yes'


In [72]:
df_overton_results.groupby(['model', 'question_id'])['represented_bool'].mean().reset_index().groupby('model').mean()

,question_id,represented_bool
model,,
gpt-3.5-turbo,619.4,0.245
gpt-4o,619.4,0.325
gpt-4o-mini,619.4,0.350
